# Plotting: Anatomy

## Single Labels

In [ ]:
import os
from surfer import Brain
%matplotlib qt4

fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
subj_dir = os.environ["SUBJECTS_DIR"]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Make brain.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

subject = "fscopy"
surf = "smoothwm"
label = 'dlpfc_1-lh'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Load files.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
## Make brain
hemi = label.split('-')[-1]
brain = Brain(subject, hemi, surf)

## Add label
f = os.path.join(fs_dir,subject,'label','april2016','%s.label' %label)
brain.remove_labels()
brain.add_label(f, color='#AFFF94', alpha=1)

if 'pfc' in label: view = 'lateral'
else: view = 'medial'

if hemi == 'lh' and view == 'lateral': brain.show_view({'azimuth': 160, 'elevation': 80}, roll=90, distance=450)
elif hemi == 'rh' and view == 'lateral': brain.show_view({'azimuth': 20, 'elevation': 80}, roll=-85, distance=450)
elif hemi == 'lh' and view == 'medial': brain.show_view({'azimuth': 10, 'elevation': 100}, roll=-85, distance=450)
elif hemi == 'rh' and view == 'medial': brain.show_view({'azimuth': 170, 'elevation': 100}, roll=90, distance=450)

In [ ]:
brain.save_image('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/%s_%s.png' %(surf,label))

## All Labels

In [ ]:
import os
from surfer import Brain
%matplotlib qt4

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Make brain.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subject_id = "fscopy"
hemi = "lh"
surf = "smoothwm"
brain = Brain(subject_id, hemi, surf)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Load files.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
subj_dir = os.environ["SUBJECTS_DIR"]

rois = ['dlpfc_1-lh', 'dlpfc_2-lh', 'dlpfc_3-lh', 'dlpfc_4-lh','dlpfc_5-lh', 'dlpfc_6-lh']
labels = [os.path.join(fs_dir,'fscopy','label','april2016','%s.label' %roi) for roi in rois]
colors = ['#ffffcc', '#c7e9b4', '#7fcdbb', '#41b6c4', '#2c7fb8', '#253494']

for n, label in enumerate(labels): 
    brain.add_label(label, color=colors[n], alpha=1)

In [ ]:
brain.save_image('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/slide7/%s_%s.png' %(surf,'group'))

# Plotting: Spectral Timecourses

## Single Labels

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
space = 'source'
model_name = 'revised'
label = 'dlpfc_5-lh'
analysis = 'stim'
freq = 'beta'
contrast = 'DBS'
baseline = (-0.5, -0.1)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
img_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

## Load source data.
npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space,analysis,label,freq)))
data = npz['data']
times = npz['times']

## Load cluster results.
f = os.path.join(root_dir, 'results', '%s_%s_frequency_results.csv' %(model_name, analysis))
clusters = read_csv(f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initialize plotting
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Intialize figure.
fig = plt.figure(figsize=(13,5), dpi=300)
ax = plt.subplot2grid((1,1),(0,0))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot DBS Difference.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

if contrast == 'DBS':
    colors = ['#0571b0','#ca0020']
    labels = ['DBS OFF','DBS ON']
elif contrast == 'Interference':
    colors = ['#7b3294','#008837']
    labels = ['Non-Conflict','Conflict']

title_dict = {'dacc-lh':'Left dACC', 'dlpfc_5-lh':'Left VLPFC', 'dlpfc_5-rh':'Right VLPFC', 'dlpfc_1-lh':'Left DLPFC'}

for m, color, legend in zip([0,1],colors,labels):

    ix, = np.where(info.DBS==m)
    mu = data[ix].mean(axis=0)
    if analysis == 'stim': mu -= mu[(times >= baseline[0])&(times <= baseline[1])].mean()
    se = data[ix].std(axis=0) / np.sqrt(len(ix))
    ax.plot(times, mu, linewidth=7, color=color, label=legend)
    ax.fill_between(times, mu-se, mu+se, color=color, alpha=0.15)

## Plot significant clusters.
for ix in np.where((clusters.Label==label)&(clusters.Freq==freq)&
                   (clusters.Contrast==contrast)&(clusters.FDR<0.05))[0]:
    
    y1, y2 = -1, 2.5
    tmin, tmax = clusters.loc[ix,'Tmin'], clusters.loc[ix,'Tmax']
    ax.fill_between(np.linspace(tmin,tmax,1e3), y1, y2, color='k', alpha=0.2) 
    ax.set_ylim(y1,y2)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Add Flourishes
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

ax.legend(loc=2, fontsize=24, markerscale=2, frameon=False, borderpad=0, handletextpad=0.2, handlelength=1.5)
ax.set_xlabel('Time (s)', fontsize=34)
ax.set_ylabel('Power (dB)', fontsize=32)
ax.tick_params(axis='both', which='major', labelsize=20)

if freq == 'theta': ax.set_title(title_dict[label], fontsize=44)
else: ax.set_title('%s %s' %(title_dict[label], freq.capitalize()), fontsize=44)
    
if analysis == 'stim':
    y1, y2 = -1, 2.5
    for x,s in zip([0, 0.4, 1.127],['IAPS','MSIT','Resp']): 
        ax.text(x+0.02,-0.75,s,fontsize=24)
        ax.vlines(x,y1,y2,linewidth=5, linestyle='--',alpha=0.4)
    ax.set_ylim(-0.8,2.2)
elif analysis == 'resp':
    y1, y2 = ax.get_ylim()
    ax.text(0.02, y1+0.05,'Resp', fontsize=24)
    ax.vlines(0.0,y1,y2,linewidth=5, linestyle='--',alpha=0.4)
    ax.set_ylim(y1,2.2)
    
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

if analysis == 'stim':
    ax.axhline(y = -0.8, linewidth=4, color='k')
    ax.axvline(x = -0.5, linewidth=4, color='k')
elif analysis == 'resp':
    ax.axhline(y = y1, linewidth=4, color='k')
    ax.axvline(x = -1.0, linewidth=4, color='k')
    
plt.tight_layout()
# plt.show()
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/%s_%s_%s_%s.png' %(label,freq,analysis,contrast))
plt.close('all')
print 'Done.'

## All labels

In [ ]:
import os
import numpy as np
import pylab as plt
import seaborn as sns
from pandas import read_csv
sns.set_style(style='white')
sns.set_context('talk')
plt.ion()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load data.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source/results/revised_stim_frequency_results.csv'
df = read_csv(f)

## Limit data.
df = df[(df.Freq=='theta') & (df.FDR<0.05)]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting: main figure.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig, ax = plt.subplots(1,1,figsize=(12,10))
colors = ['#e41a1c', '#4daf4a']

labels = ['racc-lh', 'dacc-lh', 'pcc-lh', 'dmpfc-lh', 'dlpfc_1-lh', 'dlpfc_2-lh', 
          'dlpfc_3-lh', 'dlpfc_4-lh', 'dlpfc_5-lh', 'dlpfc_6-lh', 
          'racc-rh', 'dacc-rh', 'pcc-rh', 'dmpfc-rh', 'dlpfc_1-rh', 'dlpfc_2-rh', 
          'dlpfc_3-rh', 'dlpfc_4-rh', 'dlpfc_5-rh', 'dlpfc_6-rh']

for n, label in enumerate(labels):
    
    for m, contrast in enumerate(['DBS','Interference']):
        
        ## Extract clusters.
        clusters = df.loc[(df.Contrast==contrast)&(df.Label==label),['Tmin','Tmax']]
        if not len(clusters): continue
        
        ## Plot clusters.
        y = len(labels) - n - m * 0.5
        for cluster in clusters.as_matrix(): 
            ax.fill_between(cluster, y-0.45, y-0.05, color=colors[m], alpha=0.8)
            
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting: details.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Add legend.
for label, color in zip(['DBS','Interference'],colors): ax.plot([],[],lw=10,color=color,label=label,alpha=0.7)
ax.legend(loc=2, frameon=False, fontsize=24, handlelength=1.25, borderaxespad=0)
    
## Add timing details.
y1, y2 = ax.get_ylim()
for x,s in zip([0, 0.4, 1.127],['IAPS','MSIT','Resp']): 
    ax.text(x+0.02,0.25,s,fontsize=20,weight='semibold')
    ax.vlines(x, y1, y2, linewidth=2.5, linestyle='--',alpha=0.2)    

## Fix x-axis.
ax.set_xlim(-0.25, 1.5)
ax.set_xlabel('Time (s)', fontsize=28)
ax.tick_params(axis='x', which='major', labelsize=16)

## Fix y-axis.
labels = ['rACC', 'dACC', 'mCC', 'dmPFC', 'SFG', 'pMFG', 'aMFG 1', 'aMFG 2', 'aIFG', 'pIFG'] * 2
ax.set_yticks(np.arange(len(labels))+0.5)
ax.set_yticklabels(labels[::-1], fontsize=16);

## Add dendrograms.
def dendrogram(ax, x, y1, y2, text):
    
    ## Parameters
    lw = 2.0
    alpha = 0.2
    
    ## Drawing
    ax.annotate('', (x, y1), xycoords='axes fraction', xytext=(x,y2), 
                arrowprops=dict(arrowstyle='-', color='k',  linewidth=lw, alpha=alpha))
    ax.annotate('', (x*1.02,y1), xycoords='axes fraction', xytext=(-1e-3,y1), 
                arrowprops=dict(arrowstyle='-', color='k',  linewidth=lw, alpha=alpha))
    ax.annotate('', (x*1.02,y2), xycoords='axes fraction', xytext=(-1e-3,y2), 
                arrowprops=dict(arrowstyle='-', color='k',  linewidth=lw, alpha=alpha))
    ax.annotate(text, (0,0), xycoords='axes fraction', xytext=(x*1.4, np.mean([y1,y2])), 
                rotation=90, fontsize=24, va='center')

dendrogram(ax, -0.1, 0, 0.495, 'Right Hemi')
dendrogram(ax, -0.1, 0.505, 1, 'Left Hemi')

plt.subplots_adjust(left=0.13, right=0.975, top=0.99)
# plt.show()
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/slide7/timecourse.png')
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/slide7/timecourse.svg')
plt.close()

## Left Hemisphere only

In [5]:
import os
import numpy as np
import pylab as plt
import seaborn as sns
from pandas import read_csv
sns.set_style(style='white')
sns.set_context('talk', font_scale=2)
plt.ion()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load data.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source/results/revised_stim_frequency_results.csv'
df = read_csv(f)

## Limit data.
df = df[(df.Freq=='theta') & (df.FDR<0.05)]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting: main figure.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig, ax = plt.subplots(1,1,figsize=(12,9))
colors = ['#e41a1c', '#4daf4a']

labels = ['racc-lh', 'dacc-lh', 'pcc-lh', 'dmpfc-lh', 'dlpfc_1-lh', 'dlpfc_2-lh', 
          'dlpfc_3-lh', 'dlpfc_4-lh', 'dlpfc_5-lh', 'dlpfc_6-lh']

## Add timing details.
for x,s in zip([0, 0.4, 1.127],['IAPS','MSIT','Resp']): 
    ax.text(x+0.01,0.1,s,fontsize=30)
    ax.vlines(x, 0, len(labels), linewidth=2.5, linestyle='--',alpha=0.2)  

for n, label in enumerate(labels[::-1]):
    
    for m, contrast in enumerate(['DBS','Interference']):
        
        ## Extract clusters.
        clusters = df.loc[(df.Contrast==contrast)&(df.Label==label),['Tmin','Tmax']]
        if not len(clusters): continue
        
        ## Plot clusters.
        y = n + m * 0.5
        for cluster in clusters.as_matrix(): 
            ax.hlines(y+0.25, cluster.min(), cluster.max(), color=colors[m], lw=24)
            ax.hlines(y+0.25, -0.25, cluster.min(), color=colors[m], alpha=0.2)
            
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting: details.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Add legend.
for label, color in zip(['DBS','Interference'],colors): ax.plot([],[],lw=10,color=color,label=label,alpha=0.7)
ax.legend(loc=1, frameon=False, fontsize=22, handlelength=1.1, borderaxespad=0)
  
## Fix x-axis.
ax.set_xlim(-0.25, 1.55)
ax.set_xlabel('Time (s)', fontsize=36)
ax.tick_params(axis='x', which='major', labelsize=24)

## Fix y-axis.
labels = ['rACC', 'dACC', 'mCC', 'dmPFC', 'SFG', 'pMFG', 'aMFG 1', 'aMFG 2', 'aIFG', 'pIFG']
ax.set_yticks(np.arange(len(labels))+0.5)
ax.set_yticklabels(labels[::-1], fontsize=30);
    

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.subplots_adjust(left=0.15, right=0.96, top=0.92, bottom=0.1)
# plt.show()
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/lefthemi.png', dpi=180)
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/lefthemi.svg', dpi=180)
plt.close()

# Plotting: Correlations

In [ ]:
import os
import numpy as np
import pylab as plt
import seaborn as sns
from pandas import read_csv
from scipy.stats import spearmanr
sns.set_style(style='white')
np.random.seed(47404)
plt.ion()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
space = 'source'
analysis = 'stim'
domain = 'frequency'
model = 'revised'
contrast = 'DBS'
fdr = 0.05

## Subject parameters.
subjects = np.array(['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2'])
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare MADRS Scores.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

## Prepare MADRS scores.
ratings = read_csv('/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior/Subject_Rating_Scales.csv')
ratings = ratings.set_index('Subject')
madrs = ratings.loc[subjects, 'MADRS_Now'] - ratings.loc[subjects, 'MADRS_Base']

## Prepare mania scores.
ratings = read_csv('/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior/Subject_Characteristics.csv')
ratings = ratings.set_index('Name')
mania = ratings.loc[subjects, 'Hypomania']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load cluster results.
results = read_csv(os.path.join(root_dir, 'results', '%s_%s_%s_results.csv' %(model,analysis,domain)))
results = results[(results.Contrast==contrast)&(results.FDR<fdr)].reset_index(drop=True)
results = results[results.Label == 'dlpfc_5-lh'] # Limit to the theta clusters.

## Load time series data.
npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space, analysis, 'dlpfc_5-lh', 'theta')))
data = npz['data']
times = npz['times']

## Compute condition differences.
delta = np.zeros(subjects.shape[0])
mask = (times >= results.Tmin.min()) & (times <= results.Tmax.min())    

for m, subject in enumerate(subjects):
    i, = np.where((info['Subject']==subject)&(info[contrast]==0))
    j, = np.where((info['Subject']==subject)&(info[contrast]==1))
    delta[m] += (data[j][:,mask].mean(axis=0) - data[i][:,mask].mean(axis=0)).mean()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot depression response.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig, axes = plt.subplots(1,2,figsize=(12,6))
r_val, p_val = spearmanr(madrs, delta)

## Iteratively add points.
for x,y,subj in zip(madrs,delta,madrs.index):

    if subj == 'M5' or subj == 'S2': color = '#e7298a'
    else: color = '#2b8cbe'
    axes[0].scatter(x,y,s=80,color=color)
    axes[0].text(x+0.25,y+0.07,subj)

## Add trend line.
x1, x2 = axes[0].get_xlim()
y1, y2 = axes[0].get_ylim()

z = np.polyfit(madrs, delta, 1)
p = np.poly1d(z)
axes[0].plot(np.linspace(x1,x2,1e3), p(np.linspace(x1,x2,1e3)), linewidth=2, color='k', alpha=0.8)
axes[0].set_xlim(x1,x2)
axes[0].set_ylim(y1,y2)

## Add flourishes.
axes[0].set_xlabel('MADRS [Post - Pre]', fontsize=24, labelpad=20)
axes[0].set_ylabel('Theta Power [On - Off]', fontsize=24, labelpad=20)
title = axes[0].set_title('Depression Response', fontsize=32)
title.set_position([0.5, 1.02])
axes[0].text(x1+0.5, y2*0.98, 'r = %0.3f\np = %0.3f' %(r_val,p_val), fontsize=18, va='top')
axes[0].tick_params(axis='both', which='major', labelsize=14)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot mania conversion
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Remove S2.
delta = delta[:-1]
mania = mania[:-1]
r_val, p_val = spearmanr(mania, delta)

## Iteratively add points.
for x,y,subj in zip(mania,delta,mania.index):

    if subj == 'M5' or subj == 'S2': color = '#e7298a'
    else: color = '#2b8cbe'
    axes[1].scatter(x,y,s=80,color=color)
    axes[1].text(x+2e-2,y+0.01,subj)

## Add trend line.
x1, x2 = axes[1].get_xlim()
y1, y2 = axes[1].get_ylim()

z = np.polyfit(mania, delta, 1)
p = np.poly1d(z)
axes[1].plot(np.linspace(x1,x2,1e3), p(np.linspace(x1,x2,1e3)), linewidth=2, color='k', alpha=0.8)
axes[1].set_xlim(x1,x2)
axes[1].set_ylim(y1,y2)

## Add flourishes.
axes[1].set_xticks([0,1])
axes[1].set_xticklabels(['\nNo History', '\nConverted'], fontsize=24)
axes[1].set_ylabel('Theta Power [On - Off]', fontsize=24, labelpad=20)
title = axes[1].set_title('Hypomania', fontsize=32)
title.set_position([0.5, 1.02])
axes[1].text(x1+0.02, y2*0.98, 'r = %0.3f\np = %0.3f' %(r_val,p_val), fontsize=18, va='top')
axes[1].tick_params(axis='y', which='major', labelsize=14)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#        

plt.tight_layout()
# plt.show()
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/slide8/correlations.png', dpi=120)
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/cmu_jan_2017/slide8/correlations.svg', dpi=120)
plt.close()